# 07/13/2021 | BC118: testing the effect of fixation 

The results of BC117 were inconclusive. I will run 4 experiments in the 1.1 section with full MERFISH runs to check the effect of different fixations. Those sections are small enough to allow a relatively fast test. Here I will use encoding probes in all samples. I will run a no-readout  control  at the beginning, along with DAPI, then perform the first hyb.  

| sample	| fixation |
|---|---|
|1	| 4%PFA|
|3	| 95% EtOH|
|5	| 100% MeOH|
|6	| 100 Acetone|



In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib notebook

In [2]:
# setup environmental variables in windows
import os
import sys
import shutil
env_p = sys.prefix  # path to the env
print("Env. path: {}".format(env_p))
new_p = ''
for extra_p in (r"Library\mingw-w64\bin",
    r"Library\usr\bin",
    r"Library\bin",
    r"Scripts",
    r"bin"):
    new_p +=  os.path.join(env_p, extra_p) + ';'
os.environ["PATH"] = new_p + os.environ["PATH"]  # set it for Python
os.putenv("PATH", os.environ["PATH"])  # push it at the OS level

# core libraries
import re
import random
import time
import math
import numpy as np
import pandas as pd

# optimizatin libraries
from scipy.optimize import curve_fit

# statistics libraries
# import statistics as st
# from scipy.stats import pearsonr

# datatype libraries
# from collections import Counter
# from collections import defaultdict 
import pickle

# plotting libraries
# import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patheffects as PathEffects

# from mpl_toolkits.axes_grid1 import make_axes_locatable
# from mpl_toolkits.axes_grid1.inset_locator import inset_axes
# from mpl_toolkits.axes_grid1.inset_locator import InsetPosition
# from mpl_toolkits.axes_grid1.inset_locator import mark_inset

# image / segmentation libraries
import PIL
from PIL import Image
from PIL import ImageColor
Image.MAX_IMAGE_PIXELS = None
import cv2
# from cellpose import utils
from cellpose import models
# from skimage import transform
from skimage import morphology # erosion, dilation, opening, closing, white_tophat, black_tophat, skeletonize, convex_hull_image, disk
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon
import matplotlib.image as mpimg

# I/O libraries
from pathlib import Path

# Machine learning libraries
# from sklearn import preprocessing 
# from sklearn.metrics import silhouette_samples
# from sklearn.neural_network import MLPClassifier
# from sklearn.metrics import roc_auc_score
# from sklearn.model_selection import train_test_split
# from sklearn.ensemble import RandomForestClassifier

# Sequence analysis libraries
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord

# # single cell analysis libraries
# import scanpy as sc
# import scanpy.external as sce
# sc.settings.verbosity = 2             # verbosity: errors (0), warnings (1), info (2), hints (3)
# sc.logging.print_header()

# read my own libraries
sys.path.insert(0,'C:/Software/python-functions/')
import systemChecks as syschk # backupNotebook
import imagedisplay as imdis 
import datareader as dr # readdax
import miscAnalysis as ma 

# spot analysis libraries
sys.path.insert(0,'C:/Software/')
import ImageAnalysis3 as ia
from ImageAnalysis3 import get_img_info, visual_tools, corrections, classes, alignment_tools, spot_tools

# notebook appearenace
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

Env. path: C:\Users\Leonardo\anaconda3\envs\scanpy_env_210121


In [3]:
expName = 'CNV015'
expDate = '210712'

In [4]:
baseDir = 'c:/Users/Leonardo/Dropbox/research/analysis/CNVs/'
remoteDir = f'x:/lsepulvedaduran/data/{expDate}_{expName}_data/'
outputDir = f'g:/analysis/CNVs/{expName}/'
sample = 1

if not os.path.isdir(outputDir):
    os.mkdir(outputDir)

# Calculate Flat field correction.

Ok, let's run the analysis for all fovs. I will use only hyb 0, and a single frame per stack.

In [5]:
outputFolder = f'{outputDir}flat_field_correction/'
if not os.path.isdir(outputFolder):
    os.mkdir(outputFolder)

for sample in [1]:#range(1,3,1):
    outputFolder = f'{outputDir}flat_field_correction/sample_0{sample}/'
    if not os.path.isdir(outputFolder):
        os.mkdir(outputFolder)

In [6]:
kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(250,250))
filterSigma = 20
filterSize = int(2*np.ceil(2*filterSigma)+1)+10
t = time.time()

channel_names = [750, 647, 488, 405]
hyb = 0
frames = list(range(0,140,4))

nFOVs = 21*14
nChannels = 4
numImages = nChannels*nFOVs
sample = 1

counter = 0
for fov in range(220,0,-1):
    for channel in range(nChannels):    
        fileName = f'{outputDir}flat_field_correction/sample_0{sample}/fcc_{channel}_{hyb:02d}_{fov:03d}.npy'

        if not os.path.isfile(fileName):
            imageStack = dr.DaxReader(f'{remoteDir}sample_0{sample}/hal_{fov:03d}.dax')

            I = imageStack.loadAFrame(frames[15]+channel)

            # tophat
            WT = cv2.morphologyEx(I, cv2.MORPH_TOPHAT, kernel)

            # Gaussian filter
            G = cv2.GaussianBlur(I-WT,(filterSize,filterSize),filterSigma)

            # save data
            np.save(fileName,G)

            # check the number of files in remote directories
            existentFiles = []
            for i in [1]:#range(1,3,1):
                existentFiles.extend(os.listdir(f'{outputDir}flat_field_correction/sample_0{i}/'))

            counter+=1

            elapsed_time = (time.time() - t) 
            print(f'{counter:>4d}. Sample {sample}, Hyb{hyb:3d}, fov{fov:4d}, channel{channel:2d}. '
                  f'Elapsed time: {elapsed_time/60:3.2f} min, '
                  f'{elapsed_time/counter:3.0f} sec per image, '
                  f'{(numImages-len(existentFiles))*elapsed_time/counter/60:3.0f} min remaining',end='\r')

In [ ]:
hyb = 0

for sample in [1]:#range(1,3,1):
    
    if sample == 1:
        microscope = 'mf3'    
    else:
        microscope = 'mf8' 
        
    outputFile1 = f'ffc_raw_{microscope}.npy'
    outputFile2 = f'ffc_num_{microscope}.npy'
    outputFile3 = f'ffc_{microscope}.npy'
    
    if not (os.path.isfile(outputFile1) and os.path.isfile(outputFile2) and os.path.isfile(outputFile3)):

        ffc = np.zeros((4,2048,2048))
        ffc_n = np.zeros(4)
        channel_name = [750, 647, 560, 405]
        for channel in range(4):
            for fov in range(130):

                fileName = f'{outputDir}flat_field_correction/sample_0{sample}/fcc_{channel}_{hyb:02d}_{fov:03d}.npy'
                
                # only use images without saturated pixels
                tmp = np.load(fileName)
#                 print(np.amax(tmp))
                if np.amax(tmp) < 10000:
                    ffc[channel,:,:] += tmp
                    ffc_n[channel] += 1
                    print(f'Hyb{hyb:3d}, fov{fov:4d}, channel {channel_name[channel]:2d}.',end='\r')
                else:
                    print(f'Hyb{hyb:3d}, fov{fov:4d}, channel {channel_name[channel]:2d} has high-intensity pixels. Skipping.')
                    
        np.save(outputFile1,ffc)  
        np.save(outputFile2,ffc_n)
        
        f = np.zeros((4,2048,2048))
        for channel in range(4):
            f[channel,:,:] = ffc[channel,:,:].squeeze()/ffc_n[channel]
            f[channel,:,:] = f[channel,:,:]/np.amax(f[channel,:,:])
        
        np.save(outputFile3,f)

Now let's see how the ffc look for each channel and previous exp. 

In [ ]:
counter = 0
for sample in [1]:#range(1,3,1):
    
    if sample == 1:
        fcc0 = np.load('c:/Users/Leonardo/Dropbox/research/analysis/CNVs/210516_CNV007_DNA_FISH/ffc_mf3.npy')
        fcc1 = np.load('ffc_mf3.npy')
    else:
        fcc0 = np.load('c:/Users/Leonardo/Dropbox/research/analysis/CNVs/210516_CNV007_DNA_FISH/ffc_mf8.npy')
        fcc1 = np.load('ffc_mf8.npy')

    channel_name = [750, 650, 560, 405]
    
    imsize = 2
    nrows = 3
    ncols = 4
    fig,ax = plt.subplots(nrows=nrows, ncols=ncols, figsize=(imsize*ncols,imsize*nrows))
    
    for ch in range(4):
        
        for i in range(3):
            if i == 0:
                f = fcc0[ch,:,:]
            elif i == 1:
                f = fcc1[ch,:,:]    
            else:
                f = np.abs(fcc1[ch,:,:]-fcc0[ch,:,:])
        
            ax[i, ch].imshow(f)
            ax[i, ch].axis('off')
            ax[i, ch].set_title(f'{channel_name[ch]} channel.\nmin = {np.amin(f):1.3f}, max = {np.amax(f):1.3f}', fontsize=8)

        
    fig.tight_layout(pad=2,w_pad=0,h_pad=1)
    if sample == 1:
        fig.suptitle('MERFISH3')
    else:
        fig.suptitle('MERFISH8')
    fig.savefig(f'j{expName}_07_{expDate}_{sample}.png')

There are similarities, Let's combine the data from previous experiments and see how things look 

In [ ]:
for sample in [1]:#range(1,3,1):
    
    if sample == 1:
        ffc_raw_0 = np.load('c:/Users/Leonardo/Dropbox/research/analysis/CNVs/210516_CNV007_DNA_FISH/ffc_raw_mf3.npy')
        ffc_num_0 = np.load('c:/Users/Leonardo/Dropbox/research/analysis/CNVs/210516_CNV007_DNA_FISH/ffc_num_mf3.npy')
        ffc_raw_1 = np.load('ffc_raw_mf3.npy')
        ffc_num_1 = np.load('ffc_num_mf3.npy')
        outputFile = 'ffc_mf3_cb.npy'
    elif sample == 2:
        ffc_raw_0 = np.load('c:/Users/Leonardo/Dropbox/research/analysis/CNVs/210516_CNV007_DNA_FISH/ffc_raw_mf8.npy')
        ffc_num_0 = np.load('c:/Users/Leonardo/Dropbox/research/analysis/CNVs/210516_CNV007_DNA_FISH/ffc_num_mf8.npy')
        ffc_raw_1 = np.load('ffc_raw_mf8.npy')
        ffc_num_1 = np.load('ffc_num_mf8.npy')
        outputFile = 'ffc_mf8_cb.npy'
    
    f = np.zeros((4,2048,2048))
    for channel in range(4):
        ffc = ffc_raw_0[channel,:,:].squeeze() + ffc_raw_1[channel,:,:].squeeze()
        ffc_n = ffc_num_0[channel] + ffc_num_1[channel]
        f[channel,:,:] = ffc/ffc_n
        f[channel,:,:] = f[channel,:,:]/np.amax(f[channel,:,:])
    
    np.save(outputFile,f)

Let's look at the different ffcs.

In [ ]:
counter = 0
for sample in [1]:#range(1,3,1):
    
    if sample == 1:
        fcc0 = np.load('c:/Users/Leonardo/Dropbox/research/analysis/CNVs/210516_CNV007_DNA_FISH/ffc_mf3.npy')
        fcc1 = np.load('ffc_mf3.npy')
        fcc2 = np.load('ffc_mf3_cb.npy')
    else:
        fcc0 = np.load('c:/Users/Leonardo/Dropbox/research/analysis/CNVs/210516_CNV007_DNA_FISH/ffc_mf8.npy')
        fcc1 = np.load('ffc_mf8.npy')
        fcc2 = np.load('ffc_mf8_cb.npy')

    channel_name = [750, 650, 560, 405]
    
    imsize = 2
    nrows = 3
    ncols = 4
    fig,ax = plt.subplots(nrows=nrows, ncols=ncols, figsize=(imsize*ncols,imsize*nrows))
    
    for ch in range(4):
        
        for i in range(3):
            if i == 0:
                f = fcc0[ch,:,:]
            elif i == 1:
                f = fcc1[ch,:,:]    
            else:
                f = fcc2[ch,:,:] 
        
            ax[i, ch].imshow(f)
            ax[i, ch].axis('off')
            ax[i, ch].set_title(f'{channel_name[ch]} channel.\nmin = {np.amin(f):1.3f}, max = {np.amax(f):1.3f}', fontsize=8)

        
    fig.tight_layout(pad=2,w_pad=0,h_pad=1)
    if sample == 1:
        fig.suptitle('MERFISH3')
    else:
        fig.suptitle('MERFISH8')
    fig.savefig(f'j{expName}_08_{expDate}_{sample}.png')

for mf3 I will use the combined, for mf8 I will use the one from this experiment

# Run cellpose for all images.

In [ ]:
outputFolder = f'{outputDir}cellpose/'
if not os.path.isdir(outputFolder):
    os.mkdir(outputFolder)

for sample in range(1,3,1):
    outputFolder = f'{outputDir}cellpose/sample_0{sample}/'
    if not os.path.isdir(outputFolder):
        os.mkdir(outputFolder)

    if sample == 1:
        ffc = np.load('ffc_mf3_cb.npy')
    else:
        ffc = np.load('ffc_mf8.npy')

    for i in range(130):
        outputFile = f'{outputFolder}mask_{i:03d}.npy'
        if not os.path.isfile(outputFile):
            imageStack = dr.DaxReader( f'{remoteDir}sample_0{sample}/hal_{i:03d}.dax' )
            I = imageStack.loadAFrame(15)
            I = I / ffc[3,:,:].squeeze()
            model_cyto = models.Cellpose(gpu=False, model_type='cyto')
            masks_cyto, flows_cyto, styles_cyto, diams_cyto = model_cyto.eval(I, channels=[0,0],diameter=80)

            np.save(outputFile, masks_cyto)

# Calculate cell contours

In [ ]:
# check if mosaics forlder exists
if not os.path.isdir(f'{outputDir}cell_contours/'):
    os.mkdir(f'{outputDir}cell_contours/')
for sample in range(1,3,1):
    if not os.path.isdir(f'{outputDir}cell_contours/sample_0{sample}/'):
        os.mkdir(f'{outputDir}cell_contours/sample_0{sample}/')

for sample in range(1,3,1): 
    for fov in range(130):
        maskFilePath = f'{outputDir}cellpose/sample_0{sample}/mask_{fov:03d}.npy'
        mask = np.load(maskFilePath)

        cellIDs = np.unique(mask)
        cellIDs = cellIDs[1:]
    #     print(f'fov {fov}, {cellIDs.size} cells.')
        for cell_id in range(cellIDs.size):

            contourFileName = f'{outputDir}cell_contours/sample_0{sample}/contour_{fov}_{cellIDs[cell_id]}.npy'
            if not os.path.isfile(contourFileName):

                 # find cell coordinates 
                x,y = np.where(mask == cellIDs[cell_id])

                # do not consider cells within 10 pixels from the border
                if not (np.any(np.concatenate((x,y)) <10) or np.any(np.concatenate((x,y)) >2038)):

                    # calculate cell contour
                    contour, hierarchy  = cv2.findContours(np.uint8(mask == cellIDs[cell_id]), cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)    
                    np.save(contourFileName,contour[0])
                    print(f'{contourFileName } created',end='\r')
            else:
                print(f'{contourFileName } already exists, skipping.',end='\r')

Lets overlay the cell borders with the images.

In [ ]:
# get image indexes
positions = pd.read_csv(f'C:/Software/imaging-settings/210526_CNV009/sample_01/positions_CNV009_10x13.txt', names=['X','Y'])
indexes = positions.copy()
indexes.X = ((positions.X-min(positions.X))/250).astype(int)
indexes.Y = ((positions.Y-min(positions.Y))/250).astype(int)

hyb_num = 0
imsize = int(np.floor(2048/9))
channel = 3

for sample in range(1,3,1):
    outputFile = f'{outputDir}mosaics/sample_0{sample}/mosaic_0_{imsize}.npy'
    
    mosaic = np.zeros((imsize*10, imsize*13))
    
    # get list of cell contours
    contoursFolder = f'{outputDir}cell_contours/sample_0{sample}/'
    cellContourList = os.listdir(contoursFolder)

    if  not os.path.isfile(outputFile):
        for fov in range(130):
            # get image
            imName = f'hal_{fov:03d}'
            selectedFrames = np.arange(0,140,4)+channel

            imReader = dr.DaxReader(f'{remoteDir}sample_0{sample}/{imName}.dax')
            image = imReader.loadAFrame(selectedFrames[15])
            print(f'{imName}.dax read.', end='\r')   
            
            # Get cell contours
            fileList = [x for x in cellContourList if int(x.split('_')[1]) == fov]

            # apply contours to image
            mask = np.zeros((2048,2048))
            for c in fileList:
                contour = np.load(f'{contoursFolder}{c}')
                contour = contour.squeeze()
                mask[contour[:,1],contour[:,0]] = 1

            mask = morphology.dilation(mask,morphology.selem.disk(5))
            mask = np.array(mask, dtype='bool')
            image[mask] = 2**16 - 1

            im = Image.fromarray(image)
            im = im.resize((imsize,imsize),resample=0)

            mosaic[indexes.Y[fov]*imsize:(indexes.Y[fov]+1)*imsize, indexes.X[fov]*imsize:(indexes.X[fov]+1)*imsize] = im

        np.save(outputFile, mosaic)  
      
    # plot mosaic overlay       
    mosaic = np.load(f'{outputFile}')
    xsize = 10
    ysize = xsize*10/13
    fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(xsize, ysize))
    vmax = [1000, 30000]
    ax.imshow(mosaic, vmin=0, vmax=vmax[sample-1], cmap='gray')
    ax.axis('off')
    for i in range(1,10,1):
        ax.axhline(imsize*i, ls='--')
    for i in range(1,13,1):
        ax.axvline(imsize*i, ls='--')

    txt = ax.text(mosaic.shape[0]/2, mosaic.shape[0]/10, f'{outputFile}', fontsize=10, va='center',ha='center',c='w')
    txt.set_path_effects([PathEffects.withStroke(linewidth=2, foreground='k')])

    fig.tight_layout(pad=0,w_pad=0,h_pad=0)
    fig.savefig(f'j{expName}_09_{expDate}_{sample}.png')

ok, the things look ok. The density I got is lower than Pu's experiments again. I need to Increase the density, or the number of fov. 

# Spot fitting

In [ ]:
# check if mosaics forlder exists
if not os.path.isdir(f'{outputDir}spots/'):
    os.mkdir(f'{outputDir}spots/')
for sample in [1]:
    if not os.path.isdir(f'{outputDir}spots/sample_0{sample}/'):
        os.mkdir(f'{outputDir}spots/sample_0{sample}/')


ffc = np.load('ffc_mf3.npy')

outputFolder = f'{outputDir}spots/'

for fov in range(100,21*14,1):

    for hyb in range(8):

        # read the image file
        if hyb == 0:
            imName = f'hal_{fov:03d}'
        else:
            imName = f'hal_{fov:03d}_{hyb-1}'
        imReader = dr.DaxReader(f'{remoteDir}sample_0{sample}/{imName}.dax')

        # for each channel
        for channel in range(2):
            outputFile = f'{outputFolder}sample_0{sample}/{imName}_{channel}.npy'

            # only run the analysis if file doest not exist already
            if not os.path.exists(outputFile) and not os.path.exists(f'{outputFile}.tmp.npy'):
                t = time.time()

                # save empty array so other workers do not work in this array
                np.save(f'{outputFile}.tmp.npy', np.array([])) 

                if hyb == 0:
                    selectedFrames = np.arange(0,140,4)+channel
                else:
                    selectedFrames = np.arange(0,105,3)+channel

                I = np.zeros((len(selectedFrames),2048,2048))

                # get all images
                for frame in range(len(selectedFrames)):
                    I[frame,:,:] = imReader.loadAFrame(selectedFrames[frame])
                    I[frame,:,:] = I[frame,:,:]/ffc[channel,:,:]

                # try running the code
                try:
                    print(f'\n\nStarting fitting of sample {sample}, fov {fov}, hyb {hyb}, channel {channel}')
                    spots = ia.spot_tools.fitting.fit_fov_image(I,channel, max_num_seeds=500, verbose=True)
                    print(f'{outputFile} created after {time.time() -t:>3.0f} seconds.')
                    print(f'saving {outputFile}...')
                    np.save(outputFile, spots)  
                    print(f'{outputFile} saved!\n')

                    # remove tmp file
                    os.remove(f'{outputFile}.tmp.npy')
                except:
                    print(f'Something went wrong. {outputFile} was not created, Skipping.')
            else:
                print(f'{outputFile} already exists. Skipping.', end='\r')



Starting fitting of sample 1, fov 100, hyb 0, channel 0
-- start fitting spots in channel:0, 92 seeded, 

C:/Software\ImageAnalysis3\External\Fitting_v4.py:199: RuntimeWarning: overflow encountered in exp
  return 2./(1+np.exp(t_))-1.


92 fitted in 65.901s.
g:/analysis/CNVs/CNV015/spots/sample_01/hal_100_0.npy created after  94 seconds.
saving g:/analysis/CNVs/CNV015/spots/sample_01/hal_100_0.npy...
g:/analysis/CNVs/CNV015/spots/sample_01/hal_100_0.npy saved!



Fitting takes ~1 min per fov channel.But I am ethernet limited, loading the data from Simon takes more than a minute. 

Ok, we have the cells and spots. First, let's compare the spot intensities for each hibrfidization run. 

In [ ]:
for sample in range(1,3,1):
    
    # read all the files in the folder
    outputFolder = f'{outputDir}spots/sample_0{sample}/'

    # create 2D list to keep the information
    ncols, nrows = (8,2)
    allSpots = [[0] * ncols for i in range(nrows)]

    # get all the existent values
    for hyb_num in range(8):
        for channel_num in range(2):        
            spotList = []
            for fov in range(0,130,1): 
                # read the image file
                if hyb_num == 0:
                    imName = f'hal_{fov:03d}'
                else:
                    imName = f'hal_{fov:03d}_{hyb_num-1}'

                fileName = f'{outputFolder}{imName}_{channel_num}.npy'

                if os.path.exists(fileName):
                    spots = np.load(fileName)
                    # print(f'{fileName} shape = ({spots.shape}).')

                    # calculate volume
                    volume = ((math.pi)**(3/2))*spots[:,0]*np.sqrt(spots[:,5]*spots[:,6]*spots[:,7])
                    volume = np.reshape(volume,(spots.shape[0],1))
                    spots = np.concatenate((spots,volume), axis=1)

                    # add channel to spotList
                    spotList.append(spots)

            combinedList =  np.concatenate(spotList)
            allSpots[channel_num][hyb_num-1] = combinedList

            print(f'Hyb: {hyb_num} Channel: {channel_num}, combinedList.shape = [{combinedList.shape[0]:7d}, {combinedList.shape[1]:3d}], {combinedList[0,0:3]}')

    # Saving the objects:
    with open(f'{outputDir}spots/sample_0{sample}/all_spots.pkl', 'wb') as f: 
        pickle.dump(allSpots, f) 

In [ ]:
channelName = ['750', '647']

for sample in range(1,3,1):

    outputFile = f'{outputDir}spots/sample_0{sample}/all_spots.pkl'
    with open(outputFile,'rb') as f: 
        allSpots = pickle.load(f)

    fig,ax = plt.subplots(nrows=2,ncols=12,figsize=(16,4))
    xmin    = [2,  0,    0,    0, 2, 0.1, 0.1, 0.1, -1, -1, 1, 2.5]
    xmax    = [5, 35, 2048, 2048, 5, 0.7, 0.7, 0.7,  1,  1, 4, 7]
    xlabels = ['height','z','x','y','background','width_z','width_x','width_y','sin_theta','sin_phi','error','volume']
    for channel in range(2):
        for hyb in range(8):
            for col in range(12):
                values = allSpots[channel][hyb][:,col]

                if (col>=1 and col<=3) or (col>=8 and col<=9):
                    xlabel = xlabels[col]
                else:
                    values = np.log10(values+1)
                    xlabel = f'log10({xlabels[col]})'

                hist, bin_edges = np.histogram(values, bins=100)
                bin_centers = bin_edges[0:-1] + (bin_edges[1] - bin_edges[0])/2   
                ax[channel,col].plot(bin_centers,hist/max(hist),lw=1,alpha=0.5)
                ax[channel,col].set_xlim([xmin[col], xmax[col]])

                if col == 0:
                    ax[channel,col].set_ylabel(f'{channelName[channel]} channel\nspot counts')
                if channel_num == 1:
                    ax[channel,col].set_xlabel(xlabels[col])
    
#     fig.suptitle(f'sample {sample}')
    fig.tight_layout(pad=3,w_pad=0,h_pad=0)  # otherwise the right y-label is slightly clipped
    plt.show()
    fig.savefig(f'j{expName}_10_{expDate}_{sample}.png')

Let's look a height and volume in more detail

In [ ]:
channelName = ['750', '647']

for sample in range(1,3,1):
    
    outputFile = f'{outputDir}spots/sample_0{sample}/all_spots.pkl'
    with open(outputFile,'rb') as f: 
        allSpots = pickle.load(f)

    fig,ax = plt.subplots(nrows=2,ncols=2,figsize=(10,4))

    xlabels = ['height','volume']
    for channel in range(2):
        for hyb in range(8):
            counter = 0
            for col in [0,11]:

                values = np.log10(allSpots[channel][hyb][:,col] + 1)
                xlabel = f'log10({xlabels[counter]})'
                ax[channel,counter].set_xlabel(xlabel)

                hist, bin_edges = np.histogram(values, bins=200)
                bin_centers = bin_edges[0:-1] + (bin_edges[1] - bin_edges[0])/2   
                ax[channel,counter].plot(bin_centers,hist/max(hist),lw=1,alpha=0.5)
                
                if counter == 0:
                    ax[channel,counter].set_xlim([2,5])
                elif counter == 1:
                    ax[channel,counter].set_xlim([3,7])

                if col == 0:
                    ax[channel,counter].set_ylabel(f'{channelName[channel]} channel\nspot counts')
                if ch == 2:
                    ax[channel,counter].set_xlabel(xlabels[counter])

                ax[channel,counter].set_yscale('log')
                counter+=1


    fig.tight_layout(pad=3,w_pad=1,h_pad=1)  # otherwise the right y-label is slightly clipped
    plt.show()
    fig.savefig(f'j{expName}_11_{expDate}_{sample}_0.png')

In previous notebooks, I was using the wrong variable for the volume.

In [ ]:
channelName = ['750', '647']

for sample in range(1,3,1):
    
    outputFile = f'{outputDir}spots/sample_0{sample}/all_spots.pkl'
    with open(outputFile,'rb') as f: 
        allSpots = pickle.load(f)

    fig,ax = plt.subplots(nrows=2,ncols=2,figsize=(10,5))

    xlabels = ['height','volume']
    for channel in range(2):
        for hyb in range(8):
            counter = 0
            for col in [0,11]:

                values = np.log10(allSpots[channel][hyb][:,col] + 1)
                xlabel = f'log10({xlabels[counter]})'
                ax[channel,counter].set_xlabel(xlabel)

                hist, bin_edges = np.histogram(values, bins=200)
                bin_centers = bin_edges[0:-1] + (bin_edges[1] - bin_edges[0])/2   
                ax[channel,counter].plot(bin_centers,(10**hyb)*hist/max(hist),lw=1,alpha=0.5)
                
                if counter == 0:
                    ax[channel,counter].set_xlim([2,5])
                elif counter == 1:
                    ax[channel,counter].set_xlim([3,7])

                if col == 0:
                    ax[channel,counter].set_ylabel(f'{channelName[channel]} channel\nspot counts')
                if ch == 2:
                    ax[channel,counter].set_xlabel(xlabels[counter])

                ax[channel,counter].set_yscale('log')
                counter+=1


    fig.tight_layout(pad=3,w_pad=1,h_pad=1)  # otherwise the right y-label is slightly clipped
    plt.show()
    fig.savefig(f'j{expName}_11_{expDate}_{sample}_1.png')

There is no obvious increase of intensity of the spots with the hyb. or there is? The tendency is clearer looking at the height. Maybe if I calculate the mean height for spots over log10(H)=3.15 for the 750 channel and log10(H)=3.66 foir the 647 channel.  

In [ ]:
# get number of prbes per region
selectedRegionsDF = pd.read_csv('selected_regions.csv')

channelName = ['750', '647']
channelColor = ['C3', 'C4']
channelThreshold = [3.15, 3.66]
ymax = [12000,22000]

for sample in range(1,3,1):
    
    outputFile = f'{outputDir}spots/sample_0{sample}/all_spots.pkl'
    with open(outputFile,'rb') as f: 
        allSpots = pickle.load(f)

    fig,ax = plt.subplots(nrows=2,ncols=2,figsize=(10,5))
    
    meanHeight = np.zeros((8,2))
    stdHeight = np.zeros((8,2))
    for channel in range(2):
        for hyb in range(8):

            idx = np.log10(allSpots[channel][hyb][:,0] + 1) > channelThreshold[channel]
            values_log = np.log10(allSpots[channel][hyb][idx,0] + 1)
            values = allSpots[channel][hyb][idx,0]

            hist, bin_edges = np.histogram(values_log, bins=10)
            bin_centers = bin_edges[0:-1] + (bin_edges[1] - bin_edges[0])/2   
            ax[channel,0].plot(bin_centers,hist/max(hist),lw=1.5,alpha=0.5)
            ax[channel,0].set_xlabel('log10(Spot height)')
            ax[channel,0].set_ylabel(f'{channelName[channel]} channel\nspot counts\nabove threshold')
            
            meanHeight[hyb,channel] = np.mean(values)
            stdHeight[hyb,channel]  = np.std(values)

        ax[channel,1].plot(selectedRegionsDF.a, meanHeight[:,channel],'o-',lw=1.5,alpha=0.5, c=channelColor[channel])
        ax[channel,1].set_xlabel('Probes per region')
        ax[channel,1].set_ylabel('Spot height (A.U.)')
        ax[channel,1].axis([50,400,0,ymax[channel]])
              
    
#                 hist, bin_edges = np.histogram(values, bins=200)
#                 bin_centers = bin_edges[0:-1] + (bin_edges[1] - bin_edges[0])/2   
#                 ax[channel,counter].plot(bin_centers,(10**hyb)*hist/max(hist),lw=1,alpha=0.5)
                
#                 if counter == 0:
#                     ax[channel,counter].set_xlim([2,5])
#                 elif counter == 1:
#                     ax[channel,counter].set_xlim([3,7])

#                 if col == 0:
#                     ax[channel,counter].set_ylabel(f'{channelName[channel]} channel\nspot counts')
#                 if ch == 2:
#                     ax[channel,counter].set_xlabel(xlabels[counter])

#                 ax[channel,counter].set_yscale('log')
#                 counter+=1


    fig.tight_layout(pad=3,w_pad=1,h_pad=1)  # otherwise the right y-label is slightly clipped
    plt.show()
    fig.savefig(f'j{expName}_12_{expDate}_{sample}.png')

Ok, let's check the 2D dimensions

In [ ]:
for sample in range(1,3,1):
    
    outputFile = f'{outputDir}spots/sample_0{sample}/all_spots.pkl'
    with open(outputFile,'rb') as f: 
        allSpots = pickle.load(f)

    fig,ax = plt.subplots(nrows=2,ncols=8,figsize=(16,4))

    channels = [750,650]

    for channel_num in range(2):
        for hyb_num in range(8):
            height = allSpots[channel_num][hyb_num][:,0]
            volume = allSpots[channel_num][hyb_num][:,5]*allSpots[channel_num][hyb_num][:,6]*allSpots[channel_num][hyb_num][:,7]
            ax[channel_num ,hyb_num].hexbin(height,volume,gridsize=50,xscale='log',yscale='log',extent=(2,5,-1,2),cmap=plt.cm.Blues,bins='log')
            if channel_num == 1:
                ax[channel_num ,hyb_num].set_xlabel('height')
            if hyb_num == 0:
                ax[channel_num ,hyb_num].set_ylabel(f'{channels[channel_num]} channel\nSpot volume\nw1*w2*w3')
            if channel_num == 0:
                ax[channel_num ,hyb_num].set_title(f'{selectedRegionsDF.a.iloc[hyb_num].astype("int")} probes', fontsize=10)

            if channel_num == 0:
                ax[channel_num, hyb_num].set_xticklabels([])
                ax[channel_num, hyb_num].set_xticks([])
            if hyb_num > 0:   
                ax[channel_num, hyb_num].set_yticklabels([])
                ax[channel_num, hyb_num].set_yticks([])

            ax[channel_num ,hyb_num].axis([1e2,1e5,1e-1,1e2])

    fig.tight_layout(pad=3,w_pad=0,h_pad=0)  # otherwise the right y-label is slightly clipped
    plt.show()
    fig.savefig(f'j{expName}_13_{expDate}_{sample}_0.png')

In [ ]:
channels = [750,650]
for sample in range(1,3,1):
    
    outputFile = f'{outputDir}spots/sample_0{sample}/all_spots.pkl'
    with open(outputFile,'rb') as f: 
        allSpots = pickle.load(f)

    fig,ax = plt.subplots(nrows=1,ncols=2,figsize=(10,5))

    for channel_num in range(2):
        height = []
        volume = []
        for hyb_num in range(8):
            height.extend(list(allSpots[channel_num][hyb_num][:,0]))
            volume.extend(list(allSpots[channel_num][hyb_num][:,5]*allSpots[channel_num][hyb_num][:,6]*allSpots[channel_num][hyb_num][:,7]))

        ax[channel_num].hexbin(height,volume,gridsize=50,xscale='log',yscale='log',extent=(2,5,-1,2),cmap=plt.cm.Blues,bins='log')
        ax[channel_num].set_xlabel('height')
        ax[channel_num].set_ylabel(f'{channels[channel_num]} channel\nSpot volume\nw1*w2*w3')
        ax[channel_num].set_title(f'{selectedRegionsDF.a.iloc[hyb_num].astype("int")} probes', fontsize=10)
        ax[channel_num].axis([1e2,1e5,1e-1,1e2])

    fig.tight_layout(pad=3,w_pad=0,h_pad=0)  # otherwise the right y-label is slightly clipped
    plt.show()
    fig.savefig(f'j{expName}_13_{expDate}_{sample}_1.png')

The separation is clearer than before, similar to what I saw in Pu's data. But there I had way more data. 

# Allocate spots to inside or outside cells

In [ ]:
# read all cellpose masks into an array.

for sample in range(1,3,1):

    cellposeFolder = f'{outputDir}cellpose/sample_0{sample}/'

    if os.path.isfile(f'{cellposeFolder}bg_masks.npy'):
        bg_masks = np.load(f'{cellposeFolder}bg_masks.npy')
        cell_masks = np.load(f'{cellposeFolder}cell_masks.npy')
    else:

        fileList = sorted([x for x in os.listdir(f'{cellposeFolder}/') if 'npy' in x])

        bg_masks = np.zeros((len(fileList),2048,2048))
        cell_masks = np.zeros((len(fileList),2048,2048))
        kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(50,50))

        for i in range(len(fileList)):
            maskFilePath = f'{cellposeFolder}{fileList[i]}'

            mask = np.load(maskFilePath)
            mask = mask>0
            bgMask = cv2.dilate(np.float32(mask),kernel,iterations = 1)
            #bgMask = morphology.dilation(mask,morphology.selem.disk(50))
            bgMask = bgMask<1

            cell_masks[i,:,:] = mask.astype(int)
            bg_masks[i,:,:] = bgMask.astype(int)

            print(maskFilePath,end='\r')

        np.save(f'{cellposeFolder}bg_masks.npy',bg_masks)
        np.save(f'{cellposeFolder}cell_masks.npy',cell_masks)

In [ ]:
for sample in range(1,3,1): 

    outputFile = f'{outputDir}spots/sample_0{sample}/combined_spot_list.pkl'
    if Path(outputFile).is_file():
        print(f'opening {outputFile} ...')
        with open(outputFile,'rb') as f: 
            combinedSpotList = pickle.load(f)
    else:  
        spotsFolder = f'{outputDir}spots/sample_0{sample}/'
        cellposeFolder = f'{outputDir}cellpose/sample_0{sample}/'
        bg_masks = np.load(f'{cellposeFolder}bg_masks.npy')
        cell_masks = np.load(f'{cellposeFolder}cell_masks.npy')

        # create list of lists for the data
        ncols, nrows = (8,2)
        combinedSpotList = [[0] * ncols for _ in range(nrows)]

        for channel_num in range(2):
            for hyb_num in range(8):

                # get dummy list to get all fovs 
                spotList = []
                for fov in range(130):
                    
                    print(f' Sample {sample}, channel {channel_num}, hyb {hyb_num}, fov {fov:>3d}', end='\r')

                    # getting cell masks
                    mask = cell_masks[fov,:,:].squeeze()
                    mask_bg = bg_masks[fov,:,:].squeeze()

                    # reading spot list for specific fov/hyb/channel 
                    if hyb_num == 0:
                        imName = f'hal_{fov:03d}'
                    else:
                        imName = f'hal_{fov:03d}_{hyb_num-1}'

                    fileName = f'{spotsFolder}{imName}_{channel_num}.npy'

                    # check if file exists
                    if os.path.exists(fileName):
                        spots = np.load(f'{fileName}')
                        spot_class = np.zeros((spots.shape[0],1))  
                        spot_int = np.zeros((spots.shape[0],1))
                        spot_vol = np.zeros((spots.shape[0],1))

                        for spot_num in range(spots.shape[0]):
                            x = spots[spot_num,3]
                            y = spots[spot_num,2]
                            spot_int[spot_num] = ((math.pi)**(3/2))*spots[spot_num,0]*np.sqrt(spots[spot_num,5]*spots[spot_num,6]*spots[spot_num,7])
                            spot_vol[spot_num] = spots[spot_num,5]*spots[spot_num,6]*spots[spot_num,7]

                            if np.any(mask[np.floor(y).astype(int):np.ceil(y).astype(int), np.floor(x).astype(int):np.ceil(x).astype(int)]):
                                spot_class[spot_num] = 1
                            if np.any(mask_bg[np.floor(y).astype(int):np.ceil(y).astype(int), np.floor(x).astype(int):np.ceil(x).astype(int)]):
                                spot_class[spot_num] = 2

                        spots_cat = np.concatenate((spots[:,[0,3,2]], spot_int, spot_vol ,spot_class),axis=1)            
                        spotList.append(spots_cat)

                # combine all arrays
                combinedSpotList[channel_num][hyb_num] = np.concatenate(spotList)

        # Saving the objects:
        with open(outputFile, 'wb') as f: 
            pickle.dump(combinedSpotList, f) 

In [ ]:
selectedRegionsDF = pd.read_csv('selected_regions.csv')

for sample in range(1,3,1): 

    outputFile = f'{outputDir}spots/sample_0{sample}/combined_spot_list.pkl'

    with open(outputFile,'rb') as f: 
        combinedSpotList = pickle.load(f)

    n_spots = np.zeros((2,8))
    for i in range(2):
        for j in range(8):
            n_spots[i,j] = combinedSpotList[i][j].shape[0]

    fig,ax = plt.subplots(ncols=1, nrows=1, figsize=(10,5))
    ax.plot(selectedRegionsDF.a, n_spots[0,:],'o-',c='C0',label='750')
    ax.plot(selectedRegionsDF.a, n_spots[1,:],'o-',c='C1',label='647')
    ax.axis([0,400,0,100000])
    ax.set_xlabel('Number of probes')
    ax.set_ylabel('Number of spots')
    ax.legend()

    fig.savefig(f'j{expName}_14_{expDate}_{sample}.png')

In [ ]:
for sample in range(1,3,1):
    
    outputFile = f'{outputDir}spots/sample_0{sample}/combined_spot_list.pkl'

    with open(outputFile,'rb') as f: 
        combinedSpotList = pickle.load(f)

    fig,ax = plt.subplots(nrows=2,ncols=8,figsize=(16,4))

    nbins = 50
    for channel_num in range(2):
        for hyb_num in range(8):

            idx_cell = combinedSpotList[channel_num][hyb_num][:,5] == 1
            idx_bg = combinedSpotList[channel_num][hyb_num][:,5] == 2
            height = np.log10(combinedSpotList[channel_num][hyb_num][:,0])
            spot_int = np.log10(combinedSpotList[channel_num][hyb_num][:,3])

            xmin = 2 
            xmax = 5

            ax[channel_num, hyb_num].hist(height[idx_cell], bins=nbins, range=(xmin,xmax),color='C0',alpha=0.3,density=True, log=True)
            ax[channel_num, hyb_num].hist(height[idx_bg],   bins=nbins, range=(xmin,xmax),color='C2',alpha=0.3,density=True, log=True)

            ax[channel_num, hyb_num].text(0.6, 0.8,f'inside',c='C0',transform=ax[channel_num, hyb_num].transAxes) # cells\n({sum(idx_cell)} spots)
            ax[channel_num, hyb_num].text(0.6, 0.65,f'outside',c='C2',transform=ax[channel_num, hyb_num].transAxes) # cells\n({sum(idx_bg)} spots)

            if channel_num == 0:
                ax[channel_num ,hyb_num].set_title(f'{selectedRegionsDF.a.iloc[hyb_num].astype("int")} probes', fontsize=10)
            ax[channel_num, hyb_num].axis([xmin,xmax,0,3])

            if channel_num == 0:
                ax[channel_num, hyb_num].set_xticklabels([])
                ax[channel_num, hyb_num].set_xticks([])
            if hyb_num > 0:   
                ax[channel_num, hyb_num].set_yticklabels([])
                ax[channel_num, hyb_num].set_yticks([])
            if channel_num == 0:
                ax[channel_num, hyb_num].axvline(3.1,ls='--', alpha=0.5)
            if channel_num == 1:
                ax[channel_num, hyb_num].axvline(3.4,ls='--', alpha=0.5)

    fig.add_subplot(111, frame_on=False)
    plt.tick_params(labelcolor="none", bottom=False, left=False)
    plt.xlabel("log10(Density)")
    plt.ylabel("Spot height")

    fig.tight_layout(pad=3,w_pad=0,h_pad=0)  # otherwise the right y-label is slightly clipped
    plt.show()
    fig.savefig(f'j{expName}_15_{expDate}_{sample}_0.png')

In [ ]:
for sample in range(1,3,1):
    
    outputFile = f'{outputDir}spots/sample_0{sample}/combined_spot_list.pkl'

    with open(outputFile,'rb') as f: 
        combinedSpotList = pickle.load(f)

    fig,ax = plt.subplots(nrows=2,ncols=8,figsize=(16,4))

    nbins = 50
    for channel_num in range(2):
        for hyb_num in range(8):

            idx_cell = combinedSpotList[channel_num][hyb_num][:,5] == 1
            idx_bg = combinedSpotList[channel_num][hyb_num][:,5] == 2
            height = np.log10(combinedSpotList[channel_num][hyb_num][:,0])
            spot_int = np.log10(combinedSpotList[channel_num][hyb_num][:,3])

            xmin = 2 
            xmax = 5

            ax[channel_num, hyb_num].hist(height[idx_cell], bins=nbins, range=(xmin,xmax),color='C0',alpha=0.3,density=True, log=False)
            ax[channel_num, hyb_num].hist(height[idx_bg],   bins=nbins, range=(xmin,xmax),color='C2',alpha=0.3,density=True, log=False)

            ax[channel_num, hyb_num].text(0.6, 0.8,f'inside',c='C0',transform=ax[channel_num, hyb_num].transAxes) # cells\n({sum(idx_cell)} spots)
            ax[channel_num, hyb_num].text(0.6, 0.65,f'outside',c='C2',transform=ax[channel_num, hyb_num].transAxes) # cells\n({sum(idx_bg)} spots)

            if channel_num == 0:
                ax[channel_num ,hyb_num].set_title(f'{selectedRegionsDF.a.iloc[hyb_num].astype("int")} probes', fontsize=10)
            ax[channel_num, hyb_num].axis([xmin,xmax,0,3])

            if channel_num == 0:
                ax[channel_num, hyb_num].set_xticklabels([])
                ax[channel_num, hyb_num].set_xticks([])
            if hyb_num > 0:   
                ax[channel_num, hyb_num].set_yticklabels([])
                ax[channel_num, hyb_num].set_yticks([])
            if channel_num == 0:
                ax[channel_num, hyb_num].axvline(3.1,ls='--', alpha=0.5)
            if channel_num == 1:
                ax[channel_num, hyb_num].axvline(3.4,ls='--', alpha=0.5)

    fig.add_subplot(111, frame_on=False)
    plt.tick_params(labelcolor="none", bottom=False, left=False)
    plt.xlabel("log10(Density)")
    plt.ylabel("Spot height")

    fig.tight_layout(pad=3,w_pad=0,h_pad=0)  # otherwise the right y-label is slightly clipped
    plt.show()
    fig.savefig(f'j{expName}_15_{expDate}_{sample}_1.png')

Ok, using x = 10 ** 3.1 and x = 10 ** 3.4 as thresholds for 750 and 650 respectively

# Calculate DAPI intensity inside the cells. 

I realize that I made a mistake in the code below, I was asigning the wrong index to each cell, so the counts vs dapi plots gave me a flat line (the average, expected from random asignment)

To run the anlysis again takes forever, because I need to read the images in the remote folder that has 1G connection. Let's copy the dapi images to the local folder, so if I need to do analysis again, I do not need to read remotely.

In [ ]:
t = time.time()

# check if mosaics forlder exists
if not os.path.isdir(f'{outputDir}dapi_stacks/'):
    os.mkdir(f'{outputDir}dapi_stacks/')

for sample in range(1,3,1): 

    if not os.path.isdir(f'{outputDir}dapi_stacks/sample_0{sample}/'):
        os.mkdir(f'{outputDir}dapi_stacks/sample_0{sample}/')

    # get the flat field correction
    if sample == 1:
        ffc = np.load('ffc_mf3_cb.npy')
    elif sample == 2:
        ffc = np.load('ffc_mf8.npy')

    for fov in range(130):

        dapiFileName = f'{outputDir}dapi_stacks/sample_0{sample}/dapi_stack_{fov:03d}.npy'

        if not os.path.isfile(dapiFileName):

            imageStack = dr.DaxReader( f'{remoteDir}sample_0{sample}/hal_{fov:03d}.dax' )

            # This step is very time variable. maybe because reading from the remote location 
            # depends on the address of the data on the disk and the usage of the connection 
            # at that particular period in time
            frames = np.arange(3,140,4)
            dapiStack = np.zeros((frames.size,2048,2048))
            for i in range(frames.size):
                I = imageStack.loadAFrame(frames[i]) 
                dapiStack[i,:,:] = I / ffc[3,:,:].squeeze() 

            np.save(dapiFileName, dapiStack)
            print(f'Saving {dapiFileName}. Elapsed time {(time.time()-t)/60:>3.1f} min.',end='\r')
        else:
            print(f'fov {fov:>3d} already processed. Skipping.', end='\r')

Now lets calculate the dapi intensity within each nucleus for each z position.

In [ ]:
t = time.time()

# check if mosaics forlder exists
if not os.path.isdir(f'{outputDir}dapi_int/'):
    os.mkdir(f'{outputDir}dapi_int/')

for sample in range(1,3,1): 

    if not os.path.isdir(f'{outputDir}dapi_int/sample_0{sample}/'):
        os.mkdir(f'{outputDir}dapi_int/sample_0{sample}/')

    for fov in range(130):
        
        # Get cell masks
        cellposeFolder = f'{outputDir}cellpose/sample_0{sample}/'
        maskFilePath = f'{cellposeFolder}/mask_{fov:03d}.npy'
        mask = np.load(maskFilePath)

        outputFileName = f'{outputDir}dapi_int/sample_0{sample}/dapi_int_{fov:03d}.npy'

        if not os.path.isfile(outputFileName):

            # Read local copy of dapi stack
            dapiStack = np.load(f'{outputDir}dapi_stacks/sample_0{sample}/dapi_stack_{fov:03d}.npy')
            
            cellIDs = np.unique(mask)
            cellIDs = cellIDs[1:] # Discard 0 (background)
            dapiIntensity = np.zeros(( max(cellIDs)+1,frames.size)) # the row index is the cell index in the mask

            for cell_id in range(cellIDs.size):

                print(f'fov = {fov:>3d}, cell = {cell_id:>3d}, Elapsed time: {(time.time()-t)/60:>3.2f} min.',end='\r')

                 # Find cell coordinates 
                x,y = np.where(mask == cellIDs[cell_id])

                # Do not consider cells within 10 pixels from the border
                if not (np.any(np.concatenate((x,y)) <10) or np.any(np.concatenate((x,y)) >2038)):

                    dapi_int = np.zeros(len(frames))
                    for z in range(frames.size):
                        Is = dapiStack[z,:,:].squeeze()
                        dapiIntensity[cellIDs[cell_id],z] = sum(Is[mask==cellIDs[cell_id]])

            np.save(outputFileName, dapiIntensity)
        else:
            print(f'fov {fov:>3d} already processed. Skipping.', end='\r')

Let's look how things look like.

In [ ]:
for sample in range(1,3,1):
    
    fig = plt.figure(figsize=(10,6))
    gs = fig.add_gridspec(5, 2)

    ax0 = fig.add_subplot(gs[:4,0])
    ax1 = fig.add_subplot(gs[:4,1])

    centerZ = []
    counter = 0
    for fov in range(130):    
        dapiFileName = f'{outputDir}dapi_int/sample_0{sample}/dapi_int_{fov:03d}.npy'
        dapiData = np.load(dapiFileName)

        for c in range(dapiData.shape[0]):
            x = np.arange(dapiData.shape[1])
            y = dapiData[c,:]
            if sum(y) > 0: 

                color_num = np.mod(counter,10)

                ax0.plot(x,y,alpha=0.1,c=f'C{color_num}')
                x_max = y.argmax()
                ax0.plot(x_max,y[x_max],'o',c=f'C{color_num}')
                centerZ.append(x_max)

                y_norm = (y - y.min())/(y.max()-y.min())
                ax1.plot(x,y_norm,alpha=0.5,c=f'C{color_num}')
                counter+=1


    ax0.set_ylabel('Total Dapi intensity (A.U.)')
    ax0.set_xticklabels([])
    ax0.set_xlim([0,34])
    
    ax1.set_xlabel('frame')
    ax1.set_ylabel('Total Dapi intensity (scaled)')
    ax1.set_xlim([0,34])

    ax2 = fig.add_subplot(gs[4,0])
    ax2.hist(centerZ, bins=40, range=(0,40),color='C0',alpha=0.3,density=True)
    ax2.set_xlabel('frame')
    ax2.set_xlim([0,34])

    fig.savefig(f'j{expName}_16_{expDate}_{sample}.png')

In [ ]:
for sample in range(1,3,1):
    filteredCellsDapi = []
    cellTotalDapi = np.zeros((130,50)) # fov,ncells

    for fov in range(130):    
        dapiData = np.load(f'{outputDir}dapi_int/sample_0{sample}/dapi_int_{fov:03d}.npy')

        for cell in range(dapiData.shape[0]):
            x = np.arange(dapiData.shape[1])
            y = dapiData[cell,:]
            if sum(y) > 0: # only keep cells with real intensity

                x_max = y.argmax()
                if x_max >=5 and x_max <=30: # discard cells whose center is in the borders of the range
                    filteredCellsDapi.append(y[x_max])
                    cellTotalDapi[fov,cell] = y[x_max]

    np.save(f'{outputDir}dapi_int/sample_0{sample}/filteredCellsDapi.npy',np.array(filteredCellsDapi))   
    np.save(f'{outputDir}dapi_int/sample_0{sample}/cellTotalDapi.npy',cellTotalDapi)    

In [ ]:

microscope = ['merfish3', 'merfish8']

fig, ax = plt.subplots(nrows=1,ncols=2,figsize=(12,4))

for sample in range(1,3,1):

    # read original and corrected dataset
    filteredCellsDapi = np.load(f'{outputDir}dapi_int/sample_0{sample}/filteredCellsDapi.npy')

    # flatten the arrays
    C = np.ravel(filteredCellsDapi)

    # remove zeros
    C = C[C>0]

    # plot histogram
    bin_counts, bin_edges = np.histogram(C, bins=100)
    bin_centers = np.array([0.5 * (bin_edges[i] + bin_edges[i+1]) for i in range(len(bin_edges)-1)])
    ax[sample-1].bar(bin_centers, bin_counts, width=bin_centers[1] - bin_centers[0], color='C0', label=r'Histogram',alpha=0.7)

    # Define fit function.
    def two_gaussians(x, A, B, mu1, sigma1, mu2, sigma2):
        return (A * np.exp(-1.0 * (x - mu1)**2 / (2 * sigma1**2)) + B * np.exp(-1.0 * (x - mu2)**2 / (2 * sigma2**2)))

    # Define fit function.
    def single_gaussian(x, A, mu, sigma):
        return (A * np.exp(-1.0 * (x - mu)**2 / (2 * sigma**2)))

    # fit double gaussian
    if sample == 1:
        p0 = [400, 50, 8.5e6, 5e5, 1.7e7, 1e6]
        xmax = 3e7
        ymax = 500
    elif sample == 2:
        p0 = [250, 26, 5.8e8, 5e7, 1.1e9, 1.1e8]
        xmax = 2e9
        ymax = 300
    
    popt, pcov = curve_fit(two_gaussians, xdata=bin_centers, ydata=bin_counts, p0=p0)

    # plot fitted function
    xspace = np.linspace(0, xmax, 1000)
    yspace = two_gaussians(xspace, *popt)
    ax[sample-1].plot(xspace, yspace, color='r', linewidth=1, label=r'2-gaussian fit')
    ax[sample-1].plot(xspace, single_gaussian(xspace, *popt[[0,2,3]]), 'r--', linewidth=1, label=f'Gaussian1\n($\\mu$ = {popt[2]:.0f}, $\\sigma$ = {popt[3]:.0f})')
    ax[sample-1].plot(xspace, single_gaussian(xspace, *popt[[1,4,5]]), 'r:', linewidth=1, label=r'Gaussian 2')

    # plot areas
    a = np.sqrt(2)
    m = popt[2]
    s = popt[3]
    ax[sample-1].fill([  m-  s,   m+  s,   m+  s,   m-  s], [0, 0, ymax, ymax], alpha=0.1, color='r')
    ax[sample-1].fill([2*m-a*s, 2*m+a*s, 2*m+a*s, 2*m-a*s], [0, 0, ymax, ymax], alpha=0.1, color='r')
    ax[sample-1].text(  m,0.9*ymax,'2n',ha='center',fontsize=12)
    ax[sample-1].text(2*m,0.9*ymax,'4n',ha='center',fontsize=12)

    ax[sample-1].axis([0,xmax,0,ymax])
    ax[sample-1].set_xlabel('Total DAPI intensity (Flat-field corrected)')
    ax[sample-1].legend(loc='right')
    areaFraction = (popt[0]*popt[3])/(popt[0]*popt[3] + popt[1]*popt[5])
    ax[sample-1].set_title(f'sample {sample}, {microscope[sample-1]}\n(2n cells)/(All cells) = {areaFraction:2.2f}')

    # fig.tight_layout(pad=3,w_pad=1,h_pad=1)  # otherwise the right y-label is slightly clipped
    # plt.show()
    fig.savefig(f'j{expName}_17_{expDate}.png')

Ok, very good. Let's move on.

In [ ]:
# check if mosaics forlder exists
if not os.path.isdir(f'{outputDir}cell_info/'):
    os.mkdir(f'{outputDir}cell_info/')

for sample in range(1,3,1): 
    
    if not os.path.isdir(f'{outputDir}cell_info/sample_0{sample}/'):
        os.mkdir(f'{outputDir}cell_info/sample_0{sample}/')

    cellTotalDapi = np.load(f'{outputDir}dapi_int/sample_0{sample}/cellTotalDapi.npy')   

    # For each channel,
    for channel in range(2):
        peakThresh = [10**3.1,10**3.4]
        for hyb in range(8):

            cell_fov   = []
            cell_ids   = []
            cell_area  = []
            cell_dapi  = []
            tot_spots  = []
            tot_above  = []
            tot_height = []

            # check that the file has not been made before. 

            fileName = f'{outputDir}cell_info/sample_0{sample}/h{hyb}_c{channel}.csv'
            if not os.path.isfile(fileName): 
                #print(f'{fileName} not found!')
                for fov in range(130):
                    # reading spot list for specific fov/hyb/channel 
                    if hyb == 0:
                        imName = f'hal_{fov:03d}'
                    else:
                        imName = f'hal_{fov:03d}_{hyb-1}'

                    spotFileName = f'{outputDir}spots/sample_0{sample}/{imName}_{channel}.npy'

                    if os.path.isfile(spotFileName):

                        spots = np.load(spotFileName)
#                         print(f'\t{spotFileName} found!')

                        # get the contours
                        contoursFolder = f'{outputDir}cell_contours/sample_0{sample}'
                        contourFileList = os.listdir(contoursFolder)
                        contourFileList = [c for c in contourFileList if int(c.split('_')[1]) == fov]
                        
#                         print(f'\t{len(contourFileList)} cells found')

                        for file_num in range(len(contourFileList)):

                            cell_id = int(re.split(r'[_.]',contourFileList[file_num])[2])

                            # read cell contour
                            contourFileName = f'{contoursFolder}/{contourFileList[file_num]}'
                            if os.path.isfile(contourFileName):
                                #print(f'reading {contourFileName}', end='\r')

                                contour = np.load(contourFileName)

                                # get indexes of spots that are inside the cell
                                indexes = []
                                for spot_idx in range(spots.shape[0]):
                                    y = spots[spot_idx,2]
                                    x = spots[spot_idx,3]
                                    result = cv2.pointPolygonTest(contour, (x,y), False) 
                                    if result == 1 or result == 0:
                                        indexes.append(spot_idx)
                                    #print(f'\t\t\tspot {spot_idx:>3d}. x={x:>6.1f}, y={y:>6.1f}', end='\r')

                                # calculate features
                                if cellTotalDapi[fov,cell_id] > 0:
                                    tot_spots.append(len(indexes))
                                    tot_above.append(sum(spots[indexes,0] > peakThresh[channel]))
                                    tot_height.append(sum(spots[indexes,0]))
                                    cell_ids.append(cell_id)
                                    cell_fov.append(fov)   
                                    cell_dapi.append(cellTotalDapi[fov,cell_id] )
                                    cell_area.append(cv2.contourArea(contour))
                            else:
                                print(f'{contourFileName } NOT found!')#,end='\r')
                    else:
                        print(f'{spotFileName} NOT found!')#,end='\r')
                        

                df = pd.DataFrame({'fov':cell_fov, 'cell_id':cell_ids, 'cell_area':cell_area, 'cell_dapi':cell_dapi, 'tot_spots':tot_spots, 'tot_over_thresh':tot_above, 'tot_height':tot_height})
                df.to_csv(fileName)
                print(f'{fileName} created.')
            else:
                print(f'{fileName} already exists. Skipping.')            

# Calculate the number of spots vs the DNA content

I will look at a single loci first

In [ ]:
meanInt = [8848918, 566995304] # mean
stdInt  = [ 947998,  61953814] # standard deviation

for sample in range(1,3,1):
    
    m = meanInt[sample-1]
    s = stdInt[sample-1]

    # get histogram of the data Add histograms of exponential and gaussian data.

    genomic_size = ['5kb', '5kb', '10kb', '10kb', '15kb', '15kb', '20kb', '20kb'] 

    selectedRegionsDF = pd.read_csv('selected_regions.csv')

    for channel in range(2):

        fig,ax = plt.subplots(nrows=3,ncols=8,figsize=(15,7))

        for hyb in range(8):

            df = pd.read_csv(f'{outputDir}cell_info/sample_0{sample}/h{hyb}_c{channel}.csv')
            
            ylabel = ['Total number of spots','Number of spots\nover threshold','Sum of spot intensities']
            yvar = ['tot_spots','tot_over_thresh','tot_height']
            xmax = 6
            ymax = [[50,10,50000],[80,10,150000]]
            X = 2*df.cell_dapi.to_numpy()/m
            for j in range(3):
                Y = df[yvar[j]].to_numpy()

                # calculate average
                xx, xs, yy, ys, n = ma.getMovingAverage(X, Y, bin_method='equal_sample_size', points_per_bin=50)

                # plot data and average
                ax[j,hyb].plot(X,Y,'o',alpha=0.05,ms=3)
                ax[j,hyb].plot(xx,yy,'k.-',ms=2,lw=1)

                # adding areas of 2n and 4n cells
                a = np.sqrt(2)
                mm = 2
                ss = 2*s/m
                ax[j,hyb].fill([  mm-ss,     mm+ss,     mm+ss,     mm-ss  ], [0, 0, ymax[channel][j], ymax[channel][j]], alpha=0.1, color='r')
                ax[j,hyb].fill([2*mm-a*ss, 2*mm+a*ss, 2*mm+a*ss, 2*mm-a*ss], [0, 0, ymax[channel][j], ymax[channel][j]], alpha=0.1, color='r')

                # calculate average for Y inside the filled area
                meanY2 = Y[(X >=     mm-ss) & (X <     mm+ss)].mean()
                meanY4 = Y[(X >= 2*mm-a*ss) & (X < 2*mm+a*ss)].mean()
                num_format = [".1f",".1f",".0f"]
                ax[j,hyb].text(  mm,0.8*ymax[channel][j],f'2n\n{meanY2:{num_format[j]}}',ha='center', fontsize=8)
                ax[j,hyb].text(2*mm,0.8*ymax[channel][j],f'4n\n{meanY4:{num_format[j]}}',ha='center', fontsize=8)

                ax[j,hyb].plot([0, xmax],[  meanY2, meanY2],'k:')
                ax[j,hyb].plot([0, xmax],[2*meanY2,2*meanY2],'k:')

                # add labels
                if hyb == 0:
                    ax[j,hyb].set_ylabel(ylabel[j])#, rotation='horizontal',va='center')
                if j == 2:
                    ax[j,hyb].set_xlabel('Ploidy')

                if j == 0:
                    ax[j,hyb].set_title(f'{int(selectedRegionsDF.a.iloc[hyb])} probes ({genomic_size[hyb]})',fontsize=10)
                ax[j,hyb].axis([0, xmax, 0, ymax[channel][j] ])

                if j < 2:
                    ax[j,hyb].set_xticklabels([])
                    ax[j,hyb].set_xticks([])
                else:
                    ax[j,hyb].set_xticks([0,2,4])
                    ax[j,hyb].set_xticklabels(['0','2','4'])

                if hyb > 0:   
                    ax[j,hyb].set_yticklabels([])
                    ax[j,hyb].set_yticks([])


        fig.tight_layout(pad=3,w_pad=0.5,h_pad=0.5)
        fig.savefig(f'j{expName}_18_{expDate}_{channel}_{sample}.png')


Ok, all the values are worse than Pu's experiment. I need to think about news experiments, before performing more complex analysis. I need to be able to recapitulate the results by Pu. 

In [ ]:
# get histogram of the data Add histograms of exponential and gaussian data.
MF = [3,8]
meanInt = [8848918, 566995304] # mean
stdInt  = [ 947998,  61953814] # standard deviation

for sample in range(1,3,1):

    ch_names = [750,650]
    ch_colors = ['C3', 'C4']

    fig,ax = plt.subplots(nrows=2,ncols=8,figsize=(15,5))

    p2n = np.zeros((2,8))
    m2n = np.zeros((2,8))
    
    m = 2
    s = 2*stdInt[sample-1]/meanInt[sample-1]

    for channel in range(2):
        for hyb in range(8):
            df = pd.read_csv(f'{outputDir}cell_info/sample_0{sample}/h{hyb}_c{channel}.csv')

            X = 2*df.cell_dapi.to_numpy()/meanInt[sample-1]
            Y = df['tot_over_thresh'].to_numpy()
            idx = (X >= (m-s))& (X <= (m+s))

            edges = np.arange(12)-0.5
            h, edges = np.histogram(Y[idx],bins=edges)
            centers = edges[1:] - (edges[1]-edges[0])/2

            ax[channel,hyb].plot(centers,h/h.sum(), '.-',c=ch_colors[channel])

            p2n[channel,hyb] = h[2]/h.sum()
            m2n[channel,hyb] = np.mean(Y[idx])

            ax[channel,hyb].axis([0,10,0,1])

            if hyb == 0:
                ax[channel,hyb].set_ylabel(f'{ch_names[channel]} channel\nProbability',fontsize=11)
            else:
                ax[channel,hyb].set_yticks([])
                ax[channel,hyb].set_yticklabels([])

            if channel == 1:
                ax[channel,hyb].set_xlabel('Spots per cell')
            else:
                ax[channel,hyb].set_xticks([])
                ax[channel,hyb].set_xticklabels([])
                ax[channel,hyb].set_title(f'{int(selectedRegionsDF.a.iloc[hyb])} probes ({genomic_size[hyb]})',fontsize=11)

            ax[channel,hyb].text(1,0.8,f'p(n = 2) = {p2n[channel,hyb]:.2f}',fontsize=10)

    fig.suptitle(f'MERFISH{MF[sample-1]}')
    fig.tight_layout(pad=3,w_pad=0.5,h_pad=0.5)
    fig.savefig(f'j{expName}_19_{expDate}_{sample}.png')

It is hard to compare, let's overlay

In [ ]:
# get histogram of the data Add histograms of exponential and gaussian data.
markers = ['.-','.:']
meanInt = [8848918, 566995304] # mean
stdInt  = [ 947998,  61953814] # standard deviation

fig,ax = plt.subplots(nrows=2,ncols=8,figsize=(15,5))

p2n = np.zeros((2,2,8))
m2n = np.zeros((2,2,8))

for sample in range(1,3,1):

    ch_names = [750,650]
    ch_colors = ['C3', 'C4']
    
    m = 2
    s = 2*stdInt[sample-1]/meanInt[sample-1]

    for channel in range(2):
        for hyb in range(8):
            df = pd.read_csv(f'{outputDir}cell_info/sample_0{sample}/h{hyb}_c{channel}.csv')

            X = 2*df.cell_dapi.to_numpy()/meanInt[sample-1]
            Y = df['tot_over_thresh'].to_numpy()
            idx = (X >= (m-s))& (X <= (m+s))

            edges = np.arange(12)-0.5
            h, edges = np.histogram(Y[idx],bins=edges)
            centers = edges[1:] - (edges[1]-edges[0])/2

            ax[channel,hyb].plot(centers,h/h.sum(), markers[sample-1],c=ch_colors[channel])

            p2n[sample-1,channel,hyb] = h[2]/h.sum()
            m2n[sample-1,channel,hyb] = np.mean(Y[idx])

            ax[channel,hyb].axis([0,10,0,1])

            if hyb == 0:
                ax[channel,hyb].set_ylabel(f'{ch_names[channel]} channel\nProbability',fontsize=11)
            else:
                ax[channel,hyb].set_yticks([])
                ax[channel,hyb].set_yticklabels([])

            if channel == 1:
                ax[channel,hyb].set_xlabel('Spots per cell')
            else:
                ax[channel,hyb].set_xticks([])
                ax[channel,hyb].set_xticklabels([])
                ax[channel,hyb].set_title(f'{int(selectedRegionsDF.a.iloc[hyb])} probes ({genomic_size[hyb]})',fontsize=10)

            ax[channel,hyb].text(1,0.9-0.1*sample,f'p(n = 2) = {p2n[sample-1,channel,hyb]:.2f}',fontsize=9)

    fig.tight_layout(pad=3,w_pad=0.5,h_pad=0.5)
    fig.savefig(f'j{expName}_20_{expDate}.png')

The results from MF3 and MF8 are almost indistinguishable, which is good. It is reproducible across microscopes. 

But overall,  things look worse than Pu's data. There with 100 probes I got p(n=2)~ 0.6 (and up to 0.8), a very big difference. 

Some things to observe, though: Where is the peak for each group?

|# probes | 65 | 101 | 143 | 193 | 263 | 298 | 357 | 398 | 
|---------|----|-----|-----|-----|-----|-----|-----|-----|
|MF3, 750 |  1 | 1   |  2  |  2  |  3  |  2  | 3   |  2  |
|MF3, 650 |  1 | 1   |  2  |  2  |  3  |  2  | 3   |  2  |
|MF8, 750 |  2 | 1   |  2  |  2  |  2  |  2  | 3   |  -  |
|MF8, 650 |  2 | 2   |  2  |  2  |  2  |  2  | 3   |  -  |

Let's make a summary plot. 

In [ ]:
markers = ['.-','.:']

selectedRegionsDF = pd.read_csv('selected_regions.csv')

fig,ax = plt.subplots(nrows=1,ncols=2,figsize=(12,4))

for sample in range(2):

    ax[0].plot(selectedRegionsDF.a,p2n[sample-1,0,:],markers[sample-1],c='C3', label='750')
    ax[0].plot(selectedRegionsDF.b,p2n[sample-1,1,:],markers[sample-1],c='C4', label='647')
    ax[0].set_xlabel('Number of probes')
    ax[0].set_ylabel('P(n = 2) ')
    ax[0].axis([50, 400, 0, 0.5])
    ax[0].legend()

    ax[1].plot(selectedRegionsDF.a,m2n[sample-1, 0,:],markers[sample-1],c='C3', label='750')
    ax[1].plot(selectedRegionsDF.b,m2n[sample-1, 1,:],markers[sample-1],c='C4', label='647')
    ax[1].set_xlabel('Number of probes')
    ax[1].set_ylabel('Average number of spots\n in nuclei')
    ax[1].axhline(2,ls='--',c='k', alpha=0.5)
    ax[1].axis([50, 400, 0, 4])
    ax[1].legend()

fig.tight_layout(pad=3,w_pad=3,h_pad=1)
fig.savefig(f'j{expName}_21_{expDate}.png')

Based on the current analysis, the optimal number of probes is in the range 150-200. What is the size of the regions?

    group 0 - ( 65) - 5 kb
    group 1 - (101) - 5 kb
    group 2 - (143) - 10 kb
    group 3 - (193) - 10 kb
    group 4 - (263) - 15 kb
    group 5 - (298) - 15 kb
    group 6 - (357) - 20 kb
    group 7 - (398) - 20 kb
    
If we compare groups 0/1, and 2/3, we see that the second (1 and 3) have a higher p2n than the previos group. That could make sense if what I am looking at here is that the results get a little better if the density of probes gets higher. However, from the other two sets of groups (5/6 and 7/8) I get pretty much the opposite. Maybe one way of interpretting this is that the two groups give different results from different reasons. Still, it seems that the pattern is not really clear. 

let's look @ the density of probes per kb

In [ ]:
markers = ['.-','.:']
nProbes = np.array([65, 101, 143, 193, 263, 298, 357, 398])
regionSize = np.array([5, 5, 10, 10, 15, 15, 20, 20])

fig,ax=plt.subplots(ncols=2,nrows=1, figsize=(10,5))
ax[0].plot(nProbes, nProbes/regionSize,'o-')
ax[0].axis([50,400,0,25])
ax[0].set_xlabel('Number of probes')
ax[0].set_ylabel('Probe density (probes / kb)')

for sample in range(2):

    ax[1].plot(nProbes/regionSize,p2n[sample-1,0,:],markers[sample-1],c='C3', label='750')
    ax[1].plot(nProbes/regionSize,p2n[sample-1,1,:],markers[sample-1],c='C4', label='647')
    ax[1].set_xlabel('Probe density (probes / kb)')
    ax[1].set_ylabel('P(n = 2) ')
#     ax[1].axis([50, 400, 0, 0.5])
    ax[1].legend()
    
fig.savefig(f'j{expName}_22_{expDate}.png')

I do not see a clear pattern. Which other parameters can explain the behavior?
   - Tm?
   - %GC?
    
I could calculate that for Pu's data and correlate with p2n for different regions.  

# Going back to CNV003

The first thing is to get the sequences for each of the regions used in the experiment analyzed in the CNV003 experiment. The information about the colors is in  'v:/20200707-IMR90_SI16-5kb/Analysis/Color_Usage.csv':


The question here is which are the sequences of these "u". I sent an email to Pu:
    
    Hi Pu, 

    I hope you are doing well. 

    I am revisiting the analysis I did of one of your experiments, particularly, the one in the /20200707-IMR90_SI16-5kb/ folder. I would like to know the sequences for the encoding probes used for that experiment. 

    Inside the folder above there is another folder named "analysis" and a file called Color_Usage.csv:

In [ ]:
colorUsageDF = pd.read_csv(f'v:/20200707-IMR90_SI16-5kb/Analysis/Color_Usage.csv')
colorUsageDF

    I am looking here for the sequences of u0, u1,  .. etc.

    I found a file called "adaptor_sequences.csv" inside the  \\10.245.74.212\Chromatin_NAS_2\Libraries\SI16\color_usage_info\ folder, with the following info:

In [ ]:
adaptorSequencesDF = pd.read_csv(f'w:/Libraries/SI16/color_usage_info/adaptor_sequences.csv')
adaptorSequencesDF.head(31)

    Does this mean that
    u0 -> NDB_784
    u1 -> NDB_755 
    u2 -> NDB_759

    etc?

    Thanks, 

    Leonardo
    
His response:


    Hi Leonardo,

    Your interpretation is correct. color_usage directs the encoding bits, then adaptor_sequence converts bits into readout sequences, the final readout sequences could be found here:
    \\10.245.74.212\Chromatin_NAS_2\Libraries\Readouts\designed_readouts_xxx.fasta


    Best,
    Pu

Ok, let's get the names


In [ ]:
NDB = np.zeros((32,3)).astype(int)
readouts = [750, 647, 561]

for hyb in range(32):
    for ch in range(3):
        NDB[hyb,ch] = int(adaptorSequencesDF[f'{readouts[ch]}_readout'].iloc[hyb].split('_')[1])
    

In [ ]:
NDB

Now let's get the sequences for the encoding probes 

In [ ]:
fasta = list(SeqIO.parse("w:/Libraries/DNA-MERFISH_Seon/SI16.fasta", "fasta"))

In [ ]:
def getGCContent(seq):
    return (seq.count('C') + seq.count('G'))/len(seq)

def getProbeSeparation(probePositionList):
    ps = np.zeros(len(probePositionList)-1)
    for i in range(ps.size):
        ps[i] = probePositionList[i+1] - probePositionList[i]
    return ps
    
numProbes = np.zeros((32,3)).astype(int)
meanGcContent = np.zeros((32,3))
meanProbeSeparation = np.zeros((32,3))
stdProbeSeparation = np.zeros((32,3))

for hyb in range(32):
    for ch in range(3):
        encodingProbeSeqs = []
        probePositions = []
        for ii in range(len(fasta)):
            if 'chr21' in fasta[ii].id and 'res5000' in fasta[ii].id:
                if NDB[hyb,ch] == int(fasta[ii].id.split(':')[-1].split('_')[1]):
                    encodingProbeSeqs.append(f'{fasta[ii].seq}'[40:80])
                    probePositions.append(int(fasta[ii].id.split('_')[6]))

        gcContent = [getGCContent(x) for x in encodingProbeSeqs]
        probeSeparation = getProbeSeparation(probePositions)
        
        # get average values
        numProbes[hyb,ch] = len(gcContent)
        meanProbeSeparation[hyb,ch] = np.mean(probeSeparation)
        stdProbeSeparation[hyb,ch] = np.std(probeSeparation)
        meanGcContent[hyb,ch] = np.mean(gcContent)


let's now compare p2n for the experiments

In [ ]:
m = 28964755/1e8   # mean
s =  2693210/1e8   # standard deviation

titles = [750,650,560]

p2n = np.zeros((32,3))

for ch in range(3):
    for hyb in range(1,32,1):
        df = pd.read_csv(f'g:/analysis/CNVs/CNV003/cell_info/H{hyb}R{hyb}_{ch}.csv')
  
        X = df.cell_dapi.to_numpy()/1e8
        Y = df['tot_over_thresh'].to_numpy()
        idx = (X >= (m-s))& (X <= (m+s))
        
        edges = np.arange(12)-0.5
        h, edges = np.histogram(Y[idx],bins=edges)
        
        p2n[hyb,ch] = (h[2]/h.sum())

Let's now compare p2n with other of the multiple values

In [ ]:
fig,ax = plt.subplots(ncols=3,nrows=3,figsize=(8,8))

xlabels = ['Number of probes', 'Mean probe separation', 'mean GC content']
channelNames = ['750','647','561']
xmin = [ 40, 40, 0.2]
xmax = [120,120, 0.6]

for xvar in range(3):
    if xvar == 0:
        x = numProbes[:,ch]
    elif xvar == 1:
        x = meanProbeSeparation[:,ch]
    elif xvar == 2:
        x = meanGcContent[:,ch]

    for ch in range(3):
        ax[xvar,ch].plot(x,p2n[:,ch],'o')
        ax[xvar,ch].set_xlabel(xlabels[xvar])
        ax[xvar,ch].set_ylabel('p2n')
        if xvar == 0:
            ax[xvar,ch].set_title(channelNames[ch])
        ax[xvar,ch].axis([xmin[xvar],xmax[xvar],0,1])

fig.tight_layout(pad=3,w_pad=1,h_pad=1)
fig.savefig(f'j{expName}_23_{expDate}.png')

There is no discernable pattern. So, there should be something else I can do to try getting higher labeling efficiencies. 

But there is something weird. going on. Are there correlations between the different channels?

In [ ]:
fig,ax = plt.subplots(ncols=3,nrows=1,figsize=(9,3))

xlabels = ['Number of probes', 'Mean probe separation', 'mean GC content']
channelNames = ['750','647','561']


for ch in range(3):
    ax[ch].plot([0,1],[0,1],'r-',alpha=1,lw=0.5)
    ax[ch].plot(p2n[:,ch],p2n[:,np.mod(ch+1,3)],'+')
    ax[ch].set_xlabel(f'p2n, {channelNames[ch]}')
    ax[ch].set_ylabel(f'p2n, {channelNames[np.mod(ch+1,3)]}')
    ax[ch].axis([0,1,0,1])

fig.tight_layout(pad=1,w_pad=0,h_pad=0)
fig.savefig(f'j{expName}_24_{expDate}.png')

There seems to be a weak correlation between the p2n in 750 and 650 channels, suggesting that there is a variation in the specific hybs. Is there a relationship between the number of the hyb and the p2n? 

In [ ]:
fig,ax = plt.subplots(ncols=1,nrows=1,figsize=(9,3))

xlabels = ['Number of probes', 'Mean probe separation', 'mean GC content']
channelNames = ['750','647','561']

x = np.arange(p2n.shape[0])
for ch in range(3):
    ax.plot(x,p2n[:,ch],'o-',label=f'{channelNames[ch]}')
    ax.set_xlabel('Hyb number')
    ax.set_ylabel('p2n')
    ax.axis([0,max(x),0,1])
ax.legend('on')
fig.tight_layout(pad=1,w_pad=0,h_pad=0)
fig.savefig(f'j{expName}_25_{expDate}.png')

No obvious change in p2n with hybridization round.

But still there is the question of why I am not getting the same results as Pu. That suggests that still I need to work in reproducibility. What are possible parameters?

    - Nicking solution concentration/time:
        This seem like an interesting posibility. I need to check where do they get the concentration of Hcl or the time. Maybe someone Has measured the relationship between Hcl and the nicking. 
    - Hybridization solution formamide concentration
        This in case I have not prepared the solutions properly. But that sounds weird. 
    - Washing times
        The same as before. Unlikely.
    - Amplification level.
        I am using the same level of amplification as Pu. But maybe another experiment I can get some additional amplification?
    - Something cell-type specific. 


I think that another simple test is to repeat his experiment, use the same chromosomes areas as in the CNV3 experiment, and see if I can recapitulate his results. 

So, I have several things to check

	1. What is the effect of Hcl in the nicking of DNA? How does that affect the labelling efficiency?
	2. What is the design that Steven Wang used? What about Alistair?
	3. What is the resolution for the current scDNAseq methods?


In [ ]:
syschk.backupNotebook(destination='C:/Software/python-notebooks-backup')